In [1]:
import re
import ast
import pandas as pd
import numpy as np
import time
from datetime import datetime

gameplay = pd.DataFrame()

clean = open('diamond_raw.txt').read().replace('\n', '')
champs = clean.split("|")

#Parse gameplay data 
for champ in champs[0:-1]:
    
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    
    temp_champ = champ
    graphs = re.findall ( 'data:(.*?)lines:', temp_champ, re.MULTILINE)

    name = temp_champ.split('^')[0]
    #print(name)
    dt = datetime.strptime(temp_champ.split('^')[4], 
                                        '                %b %Y            ')
    release = time.mktime(dt.timetuple())

    s = graphs[0]
    data_pop = ast.literal_eval(s[1:-25])

    data = np.array(data_pop)
    df1['date'] = data[:,0]/1000
    df1['popularity'] = data[:,1]

    s = graphs[1]
    data_wr = ast.literal_eval(s[1:-25])
    data2 = np.array(data_wr)
    df2['date'] = data2[:,0]/1000
    df2['winrate'] = data2[:,1]

    s = graphs[2]
    data_br = ast.literal_eval(s[1:-25])
    data3 = np.array(data_br)
    df3['date'] = data3[:,0]/1000
    df3['banrate'] = data3[:,1]

    df = pd.merge(pd.merge(df1, df2, on='date'), df3, on='date', how='outer')
    df['name'] = name
    df['release'] = release
    gameplay = pd.concat([gameplay, df], axis=0)
    
gameplay.sort_values('date', inplace=True)
gameplay.reset_index(drop=True, inplace = True)
# gameplay.head()

In [2]:
# Parse patch data

dfp = pd.read_csv('patchnotes.txt', sep='|', names=['patch', 'date', 'buffs', 'nerfs'])
dfp.fillna(' ', inplace=True)
dfp = dfp[dfp['buffs'].str.contains('buffs')]
dfp['buffs'] = dfp['buffs'].str.replace('buffs:', '')
dfp['buffs'] = dfp['buffs'].str.replace('[:;.]+', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('nerfs:', '')
dfp['nerfs'] = dfp['nerfs'].str.replace('[:.;]+', '')
dfp.sort_values('date', inplace=True)
dfp.reset_index(drop=True, inplace = True)
# dfp.head()

changes = pd.DataFrame(np.nan, index=range(0,len(gameplay)), columns=['changes'])

for index, row in dfp.iterrows():
    if index != 0:
#         print(dfp['date'].iloc[index-1])
#         print(row['date'])
        for champ in dfp.iloc[index]['buffs'].split(','):
            idx = gameplay[(gameplay['date'] < row['date']) & 
                     (gameplay['date'] >= dfp['date'].iloc[index-1]) &
                    (gameplay['name']==champ)].index.tolist()
            changes.loc[idx] = 'buff'
        for champ in dfp.iloc[index]['nerfs'].split(','):
            idx = gameplay[(gameplay['date'] < row['date']) & 
                     (gameplay['date'] >= dfp['date'].iloc[index-1]) &
                    (gameplay['name']==champ)].index.tolist()
            changes.loc[idx] = 'nerf'

changes.replace(np.nan, 'nochange', inplace=True)
# changes.describe()
# changes.head()

df = pd.concat([gameplay, changes], axis=1)


In [3]:
df.to_csv('loldata_diamondonly.csv', index=False)